In [1]:
import subprocess
import pandas as pd
import os

In [2]:
from eval import calculate_nad, calculate_metrics
import joblib
import json
from adat.models.classification_model import LogisticRegressionOnTfIdf

In [3]:
from sklearn.model_selection import ParameterSampler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import numpy as np
import itertools
import time

In [89]:
#dataset = 'kaggle_transactions'
#dataset = 'ag_news' 
dataset = 'insurance'
#dataset = 'ai_academy_data'


base_dir = f'results/{dataset}'


In [90]:
result = []
for method in ['random']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            try:
                args = json.load(open(os.path.join(base, folder, 'args.json')))
                metrics = json.load(open(os.path.join(base, folder, 'eval_metric.json')))
                args.update(metrics)
                result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD', 
                                                   'NAD_new',
                                                    'beam_size', 'num_steps', 'std']})
            except Exception as e:
                print(e)
result = pd.DataFrame(result)

In [91]:
print('random')
result[(result['beam_size'] == 30)&(result['num_steps'] == 30)].sort_values(by='NAD_new', ascending=False).head()

random


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,beam_size,num_steps,std
0,0.151515,0.1,0.021645,3.86,0.053367,0.066386,30,30,0.10
4,0.151515,0.1,0.021645,3.86,0.053367,0.066386,30,30,0.50
6,0.151515,0.1,0.021645,3.86,0.053367,0.066386,30,30,0.01
7,0.151515,0.1,0.021645,3.86,0.053367,0.066386,30,30,0.20
9,0.151515,0.1,0.021645,3.86,0.053367,0.066386,30,30,1.00


In [92]:
print('random')
result.sort_values(by='NAD_new', ascending=False)

random


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,beam_size,num_steps,std
0,0.151515,0.10,0.021645,3.86,0.053367,0.066386,30,30,0.10
4,0.151515,0.10,0.021645,3.86,0.053367,0.066386,30,30,0.50
6,0.151515,0.10,0.021645,3.86,0.053367,0.066386,30,30,0.01
7,0.151515,0.10,0.021645,3.86,0.053367,0.066386,30,30,0.20
9,0.151515,0.10,0.021645,3.86,0.053367,0.066386,30,30,1.00
11,0.151515,0.10,0.021645,3.86,0.053367,0.066386,30,30,0.05
1,0.020202,0.02,0.000601,1.90,0.011223,0.020106,1,100,1.00
2,0.020202,0.02,0.000601,1.90,0.011223,0.020106,1,100,0.50
3,0.020202,0.02,0.000601,1.90,0.011223,0.020106,1,100,0.01
5,0.020202,0.02,0.000601,1.90,0.011223,0.020106,1,100,0.10


In [93]:
xs = os.listdir(f'results/{dataset}/hotflip')
x = np.sort(xs)[-1]
result = json.load(open(f'results/{dataset}/hotflip/{x}/eval_metric.json', 'r'))
result = pd.DataFrame([result])

In [94]:
print('hotflip')
result[['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD', 'NAD_new']]

hotflip


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new
0,0.171717,0.7,0.31485,5.45,0.130082,0.0


In [95]:
result = []
for method in ['mcmc']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            try:
                args = json.load(open(os.path.join(base, folder, 'args.json')))
                metrics = json.load(open(os.path.join(base, folder, 'eval_metric.json')))
                args.update(metrics)
                result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                                    'NAD_new', 
                                                   'beam_size', 'num_steps', 'std', 'sigma_wer', 'sigma_class']})
            except Exception as e:
                print(e)
result = pd.DataFrame(result)

In [96]:
print('mcmc')
result[(result['WER'] < 2)&(result['beam_size'] == 30)&(result['num_steps'] == 30)].sort_values(by='NAD_new', ascending=False).head()

mcmc


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,beam_size,num_steps,std,sigma_wer,sigma_class
4,0.040404,0.08,0.010489,0.54,0.036195,0.044837,30,30,0.05,1.00,0.2
33,0.070707,0.06,0.008265,0.66,0.025589,0.038945,30,30,1.00,1.50,0.2
21,0.070707,0.04,0.003899,0.54,0.023569,0.032772,30,30,0.01,1.00,0.2
30,0.070707,0.05,0.006389,0.39,0.020539,0.031313,30,30,0.05,1.50,0.2
40,0.060606,0.05,0.006223,0.32,0.023569,0.027946,30,30,0.05,0.75,0.2


In [97]:
print('mcmc')
result[(result['WER'] < 2)].sort_values(by='NAD_new', ascending=False).head()

mcmc


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,beam_size,num_steps,std,sigma_wer,sigma_class
4,0.040404,0.08,0.010489,0.54,0.036195,0.044837,30,30,0.05,1.00,0.2
33,0.070707,0.06,0.008265,0.66,0.025589,0.038945,30,30,1.00,1.50,0.2
21,0.070707,0.04,0.003899,0.54,0.023569,0.032772,30,30,0.01,1.00,0.2
30,0.070707,0.05,0.006389,0.39,0.020539,0.031313,30,30,0.05,1.50,0.2
40,0.060606,0.05,0.006223,0.32,0.023569,0.027946,30,30,0.05,0.75,0.2


In [98]:
result = []
for method in ['cascada']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            args = json.load(open(os.path.join(base, folder, 'args.json')))
            metrics = json.load(open(os.path.join(base, folder, 'eval_metric.json')))
            args.update(metrics)
            result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                               'NAD_new',
                                               'levenshtein_weight', 'beam_size', 'max_steps', 'learning_rate']})
result = pd.DataFrame(result)

In [99]:
print('cascada')
result[(result['WER'] < 2)&(result['beam_size'] == 30)&(result['max_steps'] == 30)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,levenshtein_weight,beam_size,max_steps,learning_rate
9,0.0,0.0,0.000635,0.48,0.0,0.0,0.1,30,30,2.0
17,0.0,0.0,-0.000144,0.06,0.0,0.0,0.1,30,30,1.0


In [100]:
print('cascada')
result[(result['WER'] < 9)&(result['beam_size'] == 30)&(result['max_steps'] == 30)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,levenshtein_weight,beam_size,max_steps,learning_rate
39,2.020202e-02,0.05,0.009766,3.68,0.008107,0.017254,0.1,30,30,4.0
37,1.010101e-02,0.05,0.007868,6.09,0.005586,0.010936,1.0,30,30,2.0
54,1.010101e-02,0.01,0.003689,3.32,0.003899,0.006592,1.0,30,30,1.0
66,5.050505e-02,0.07,0.014821,8.83,0.003704,0.003190,1.0,30,30,4.0
18,1.010101e-02,0.02,0.002105,6.18,0.003899,0.000532,10.0,30,30,4.0
9,0.000000e+00,0.00,0.000635,0.48,0.000000,0.000000,0.1,30,30,2.0
17,0.000000e+00,0.00,-0.000144,0.06,0.000000,0.000000,0.1,30,30,1.0
22,1.110223e-16,0.01,0.003185,7.47,0.000505,0.000000,10.0,30,30,1.0
59,1.110223e-16,-0.01,0.000435,6.60,0.000000,0.000000,10.0,30,30,2.0


In [101]:
print('cascada')
result[(result['WER'] < 2)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,levenshtein_weight,beam_size,max_steps,learning_rate
35,0.000000,0.01,0.002016,1.17,0.010101,0.010101,0.1,1,5,2.0
64,0.000000,0.01,0.002630,1.34,0.010101,0.010101,0.1,1,30,2.0
63,0.020202,0.03,0.006919,0.50,0.020202,0.010101,0.1,1,100,1.0
16,0.020202,0.03,0.010309,1.58,0.030303,0.010101,0.1,1,100,2.0
53,0.000000,0.01,0.000940,0.46,0.010101,0.010101,0.1,1,30,1.0
38,0.000000,0.00,0.000873,0.61,0.000000,0.000000,0.1,30,100,2.0
67,0.000000,0.00,0.000144,0.33,0.000000,0.000000,0.1,1,5,1.0
58,0.000000,0.00,-0.000193,0.06,0.000000,0.000000,0.1,30,5,1.0
52,0.000000,0.00,0.000384,0.38,0.000000,0.000000,0.1,30,5,2.0
45,0.000000,0.00,0.000666,0.39,0.000000,0.000000,0.1,100,30,2.0


In [102]:
print('cascada')
result[(result['WER'] < 9)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,levenshtein_weight,beam_size,max_steps,learning_rate
79,0.070707,0.09,0.020937,6.16,0.013591,0.027454,0.1,1,100,4.0
36,0.070707,0.07,0.020765,6.09,0.009719,0.022404,0.1,1,30,4.0
11,0.060606,0.07,0.017762,5.22,0.008596,0.020240,0.1,1,5,4.0
29,0.020202,0.04,0.008926,3.73,0.007602,0.017254,0.1,30,100,4.0
60,0.020202,0.04,0.008751,3.44,0.007602,0.017254,0.1,30,5,4.0
...,...,...,...,...,...,...,...,...,...,...
45,0.000000,0.00,0.000666,0.39,0.000000,0.000000,0.1,100,30,2.0
47,0.000000,-0.01,0.000221,2.60,0.000000,0.000000,1.0,1,5,1.0
48,0.000000,-0.01,0.000649,6.70,0.000000,0.000000,10.0,30,100,2.0
49,0.010101,0.00,0.002404,5.89,0.000505,0.000000,10.0,1,5,2.0
